In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from timm import create_model
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import copy

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(root='C:/Users/liamcee/Documents/farbruh/fer13/train', transform=transform)
test_dataset = datasets.ImageFolder(root='C:/Users/liamcee/Documents/farbruh/fer13/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, num_classes)

cuda 7


In [3]:
model = create_model(
    'convnext_tiny',
    pretrained=True,
    num_classes=num_classes
)
model = model.to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=4, gamma=0.1)

In [5]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Training")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

def evaluate(model, loader, criterion, epoch):
    global best_acc, best_model_wts
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Evaluating")

    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)

    print(f"Test  Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, 'best_convnext_fer13.pth')
        print(f"best model saved with accuracy: {best_acc:.2f}%")

In [6]:
epochs = 20
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    start_time = time.time()
    
    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion, epoch)
    
    scheduler.step()

    elapsed = time.time() - start_time
    print(f"Time: {elapsed:.2f} s")


Epoch 1/20


Training: 100%|██████████| 1795/1795 [06:48<00:00,  4.39it/s, acc=53.4, loss=1.13] 


Train Loss: 1.2154, Accuracy: 53.36%


Evaluating: 100%|██████████| 449/449 [00:32<00:00, 13.61it/s, acc=63.1, loss=0.33]  


Test  Loss: 0.9755, Accuracy: 63.07%
best model saved with accuracy: 63.07%
Time: 441.96 s

Epoch 2/20


...

Test  Loss: 2.5003, Accuracy: 70.34%
Time: 1078.55 s

Epoch 20/20


Training: 100%|██████████| 1795/1795 [17:08<00:00,  1.75it/s, acc=99.8, loss=0.000531]


Train Loss: 0.0038, Accuracy: 99.84%


Evaluating: 100%|██████████| 449/449 [01:05<00:00,  6.87it/s, acc=70.3, loss=2.6]     

Test  Loss: 2.5017, Accuracy: 70.33%
Time: 1093.67 s


In [7]:
metrics = {
    'epoch': list(range(1, epochs + 1)),
    'train_loss': train_losses,
    'train_accuracy': train_accuracies,
    'test_loss': test_losses,
    'test_accuracy': test_accuracies
}

df_metrics = pd.DataFrame(metrics)

df_metrics.to_csv("convnext_fer13_metrics.csv", index=False)

torch.save(model.state_dict(), 'ferconvnext_weights.pth')
torch.save(model, 'ferconvnext_full.pth')